In [1]:
import pandas as pd
import glob
import numpy as np
import json
from haversine import haversine
import datetime
import os

In [2]:
zip_centers=json.load(open("/home/jian/Docs/Geo_mapping/updated_zip_centers_JL_2019-05-23.json","r"))

In [3]:
df_store_list=pd.read_table("/home/jian/BigLots/static_files/Store_list/MediaStormStores20200301-134228-900.txt",
                            sep="|",dtype=str,usecols=['location_id','latitude_meas','longitude_meas'])
df_store_list=df_store_list[df_store_list['location_id']!="6990"]
df_store_list=df_store_list[df_store_list['location_id']!="145"]
df_store_list=df_store_list[['location_id','latitude_meas','longitude_meas']]
df_store_list['latitude_meas']=df_store_list['latitude_meas'].astype(float)
df_store_list['longitude_meas']=df_store_list['longitude_meas'].astype(float)
df_store_list['store_coor']=df_store_list[['latitude_meas','longitude_meas']].values.tolist()
dict_store_coor=df_store_list.set_index("location_id").to_dict()['store_coor']

In [4]:
TA_zips=pd.ExcelFile("/home/jian/Projects/Big_Lots/New_TA/TA_created_in_201906/final_output_20190718/BL_final_TA_updated_JL_2019-07-18.xlsx")
TA_zips=TA_zips.parse("view_by_store",dtype=str)

In [5]:
df_temporary=TA_zips[['location_id','trans_P_zips_70_within_TA','trans_S_zips_70_within_TA','zips_in_10']]
df_zip_by_store=pd.DataFrame()

for ind,row in df_temporary.iterrows():
    location_id=row['location_id']
    P_zips=eval(row['trans_P_zips_70_within_TA'])
    S_zips=eval(row['trans_S_zips_70_within_TA'])
    zip_10=eval(row['zips_in_10'])
    

    df_P=pd.DataFrame(zip([location_id]*len(P_zips),P_zips))
    if len(df_P)>0:
        df_P.columns=['location_id','zip_cd']
        df_P['zip_type']="P"
        
    df_S=pd.DataFrame(zip([location_id]*len(S_zips),S_zips))
    if len(df_S)>0:
        df_S.columns=['location_id','zip_cd']
        df_S['zip_type']="S"
    
    df_10=pd.DataFrame(zip([location_id]*len(zip_10),zip_10))
    if len(df_10)>0:
        df_10.columns=['location_id','zip_cd']
        df_10['zip_type']="zip_10"
    
    df_zip_by_store=df_zip_by_store.append(df_P).append(df_S).append(df_10)

In [6]:
df_new_stores=pd.merge(df_store_list,df_zip_by_store,on="location_id",how="left")
df_new_stores=df_new_stores[pd.isnull(df_new_stores['zip_cd'])]

df_existing_stores=pd.merge(df_store_list,df_zip_by_store,on="location_id",how="left")
df_existing_stores=df_existing_stores[pd.notnull(df_existing_stores['zip_cd'])]


In [7]:
df_new_stores.shape,df_new_stores['location_id'].nunique()

((31, 6), 31)

In [8]:
del df_new_stores['zip_cd']
del df_new_stores['zip_type']


In [9]:
df_new_store_zips=pd.DataFrame()
for ind,row in df_new_stores.iterrows():
    store_id=row['location_id']
    store_center=row['store_coor']
    list_zip_per_store=[]
    for zip_cd,v in zip_centers.items():
        dist=haversine(store_center,v,unit="mi")
        if dist<=10:
            list_zip_per_store.append(zip_cd)
    df=pd.DataFrame({"zip_cd":list_zip_per_store})
    df['location_id']=store_id
    df['zip_type']="zip_10"
    df_new_store_zips=df_new_store_zips.append(df)
print(df_new_store_zips.shape,df_new_store_zips['location_id'].nunique(),df_new_store_zips['zip_cd'].nunique())
df_new_store_zips.head(2)

(775, 3) 31 775


,zip_cd,location_id,zip_type
0,48919,4679,zip_10
1,48912,4679,zip_10


In [10]:
df_new_stores=pd.merge(df_new_stores,df_new_store_zips,on="location_id",how="left")
df_new_stores.head(2)

,location_id,latitude_meas,longitude_meas,store_coor,zip_cd,zip_type
0,4679,42.723314,-84.424187,"[42.723314, -84.424187]",48919,zip_10
1,4679,42.723314,-84.424187,"[42.723314, -84.424187]",48912,zip_10


In [11]:
df_existing_stores.head(2)

,location_id,latitude_meas,longitude_meas,store_coor,zip_cd,zip_type
0,3,33.42157,-82.018921,"[33.42157, -82.018921]",30906,P
1,3,33.42157,-82.018921,"[33.42157, -82.018921]",30904,S


In [13]:
df_output=df_new_stores.append(df_existing_stores)
df_output['location_id']=df_output['location_id'].astype(int)
df_output=df_output.sort_values(['location_id','zip_type','zip_cd'])

In [15]:
df_output.to_csv("./BL_zips_in_TA_for_stores_open_2020March_JL_"+str(datetime.datetime.now().date())+".csv",index=False)